# LUS-CS433

## 1. Code Exploration

### Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
import seaborn as sns
import glob

### Downloading data

## 2. Data Exploration

In [3]:
"""# getting the list of names
!ls LUS\ images/negative > neg_names.csv
!ls LUS\ images/positive > pos_names.csv

neg = pd.read_csv('neg_names.csv', header=None)
pos = pd.read_csv('pos_names.csv', header=None)

data = neg.append(pos, ignore_index=True)
data

pos = pd.read_csv('pos_names.csv', header=None)

# removing .png extension
pos[0] = pos[0].apply(lambda x: x[:-4])

# extracting site
pos[0] = pos[0].apply(lambda x: x.split('_')[2])

pos

sums = pd.get_dummies(pos).sum()

plt.bar(sums.index, sums.values)
plt.xlabel('Site')
plt.ylabel('Count')
plt.tight_layout()"""

"# getting the list of names\n!ls LUS\\ images/negative > neg_names.csv\n!ls LUS\\ images/positive > pos_names.csv\n\nneg = pd.read_csv('neg_names.csv', header=None)\npos = pd.read_csv('pos_names.csv', header=None)\n\ndata = neg.append(pos, ignore_index=True)\ndata\n\npos = pd.read_csv('pos_names.csv', header=None)\n\n# removing .png extension\npos[0] = pos[0].apply(lambda x: x[:-4])\n\n# extracting site\npos[0] = pos[0].apply(lambda x: x.split('_')[2])\n\npos\n\nsums = pd.get_dummies(pos).sum()\n\nplt.bar(sums.index, sums.values)\nplt.xlabel('Site')\nplt.ylabel('Count')\nplt.tight_layout()"

## 3. Creating Datasets

### 3.1 Creating file folders

In [153]:
"""! mkdir data/LUS_proc_images
! mkdir data/LUS_proc_images/train
! mkdir data/LUS_proc_images/val
! mkdir data/LUS_proc_images/train/true
! mkdir data/LUS_proc_images/train/false
! mkdir data/LUS_proc_images/val/true
! mkdir data/LUS_proc_images/val/false """

! mkdir data/butter_proc_images
! mkdir data/butter_proc_images/train
! mkdir data/butter_proc_images/val
! mkdir data/butter_proc_images/train/true
! mkdir data/butter_proc_images/train/false
! mkdir data/butter_proc_images/val/true
! mkdir data/butter_proc_images/val/false 

### 3.2 Image preprocessing

In [6]:
# Main mask used to capture the relevant portion of LUS images. Crafted manually. Is [1,1,1] where image is relevant

# Image dimensions
nb_rows = 1080
nb_cols = 792
nb_channels = 3

mask = np.zeros([nb_rows, nb_cols, nb_channels])

# Filling mask
for row in range(nb_rows):
    for col in range(nb_cols):
        # Delimitations of the cone like portion of a LUS image
        if row > 25 and row < 1010 and col < 762 and (-4/5 * row + 293) < col and (4/5*row) + nb_cols-293 > col:
            mask[row, col] = [1,1,1]

We then create a function that will runs over our expert images, crops them, and organises them in a folder architecture that will later be used through the PyTorch DataLoader class.

In [162]:
def create_samples(nb_samples, ratio, path):
    """
    Crops nb_spamples images from the given folder and 
    organises them in training and validation folders, in "true" labeled subfolders
    """
    images_path = glob.glob(path)
    # TODO: do a random split mechanism ?
    train = int(nb_samples * ratio)
        
    for i, image in enumerate(images_path[:nb_samples]):
        cv2_img = cv2.imread(image)
        masked_img = cv2.resize(cv2_img, (nb_cols, nb_rows))*mask792
        # Selecting image name from path name
        im_name = image.split("/")[-1]
        
        if i >= train:
            cv2.imwrite(f"data/butter_proc_images/val/true/{im_name}", masked_img)
        else:
            success = cv2.imwrite(f"data/butter_proc_images/train/true/{im_name}", masked_img)

In [163]:
# Make sure that the files are empty
"""! rm -rfv data/LUS_proc_images/val/true/*
! rm -rfv data/LUS_proc_images/train/true/*"""

# Generates the samples!
nb_samples = 3455
ratio = 0.8
path = "data/Ultrason butterflynetwork/*.png"
create_samples(nb_samples, ratio, path)

'#Make sure that the files are empty\n! rm -rfv data/LUS_proc_images/val/true/*\n! rm -rfv data/LUS_proc_images/train/true/*'

In [9]:
#CHECK THAT INDEED ALL IMAGES HAVE THE SAME SHAPE
path = "data/LUS_proc_images/train/true/*.png"
images_path = glob.glob(path)
for image in images_path:
    x = cv2.imread(image)
    if(x.shape != (1080,792,3)):
        print(x.shape)
        print(image)

KeyboardInterrupt: 

In [ ]:
def compute_mean_std(img, mask):
    len_x,len_y, _ = img.shape
    mean = np.mean(img)*(1080*792/active_792)
    var = 0
    for x in range(len_x):
        for y in range(len_y):
            if(mask[x,y,0]==1):
                var += (img[x,y,0] - mean)**2
    
    var /= active_792 
    
    return mean, np.sqrt(var)
            

In [ ]:
path = "data/LUS_proc_images/train/true/2_QPIG_1.png"
img = cv2.imread(path)

mean = cv2.mean(img)[0]*(1080*792/active_792)
mean2 = np.mean(img)

print(mean, mean2, mean2/mean)

#### Compute mean brightness over all sample

In [ ]:


def create_brightness_df(path):
    df = pd.DataFrame(columns = ['mean', 'std'])
    images_path = glob.glob(path)
    for i,image in enumerate(images_path):
        img = cv2.imread(image)
        m, s = compute_mean_std(img,mask792)
        df.loc[i] = [m, s]
        
    return df

In [ ]:
df = create_brightness_df("data/LUS_proc_images/train/true/*.png")

In [ ]:
df.to_csv("mean_std.csv")

In [ ]:
df.boxplot()

In [ ]:
df['mean'].hist()

In [ ]:
df['std'].hist()

In [ ]:
df.describe()

In [ ]:
df[(df['mean']<15) & (df['std']<32)].hist()

In [ ]:
df2 = df.astype(int)
df2 = pd.crosstab(df2['mean'], df2['std'])
sns.heatmap(df2)

In [ ]:
MEAN_UP, MEAN_DOWN = 7, 120  #mean_down, mean_up (based around the described df frame)
LAST_WRITE_NB_FRAMES = 20

In [8]:
# TODO comment
def split_video(video_path):
    video = cv2.VideoCapture(video_path)
    i = 0
    df_video = pd.DataFrame(columns = ['mean', 'std'])
    while(video.isOpened()):
        ret, image = video.read()
        if ret == True:
            video.set(cv2.CAP_PROP_POS_MSEC,(i*1000))
            image = image * mask
            m, s = compute_mean_std(image, mask)
            df_video.loc[i] = [m[0,0], s[0,0]]
            i += 1
        else:
            break
    video.release()
    return df_video

In [ ]:
#Creates samples from a video
def create_samples_from_video(video_path, folder_path):
    video = cv2.VideoCapture(video_path)
    i = 0
    last_write = 100 #variable to avoid writing 2 images that are almost the same 
    
    while(video.isOpened()):
        ret, image = video.read()
        if ret == True:
            #if we have access to an image, we check the mean, after applying the mask
            image = image * mask792
            m = np.mean(image) *(1080*792/ active_792)
            
            if (m > MEAN_UP or m < MEAN_DOWN) and (last_write > LAST_WRITE_NB_FRAMES) :
                cv2.imwrite(f"{folder_path}/{video_path[31:-4]}_{i}.png", image)
                i+=1
                last_write = 0
            
            last_write += 1
        else:
            #need to get out the while loop if we can't read a file
            break
    video.release()

In [ ]:
def multiple_videos_sample(path,nb_videos,ratio):
    videos_path = glob.glob(path)
    videos_in_train = int(nb_videos * ratio)
    #Creates first video in the train folder
    for video_path in videos_path[:videos_in_train]:
        create_samples_from_video(video_path, "data/LUS_proc_images/train/false")
    
    #then in the val folder
    for video_path in videos_path[videos_in_train:nb_videos]:
        create_samples_from_video(video_path, "data/LUS_proc_images/val/false")

In [ ]:
multiple_videos_sample("data/Ultrason butterflynetwork/*.mp4", 50, 0.8)

In [ ]:
"""# Renaming to appropriate format
def replace_dot_by_underscore(path):
    path_list = glob.glob(path)
    
    for old_path in path_list:
        new_path = old_path.replace(".", "_", 1)
        os.rename(old_path, new_path)
        
replace_dot_by_underscore("data/Ultrason butterflynetwork/*")"""

In [ ]:
df_split = split_video("data/Ultrason butterflynetwork/1_1_QAID_1.mp4")

In [ ]:
df_split

In [ ]:
df_split = split_video("data/Ultrason butterflynetwork/1_9_QLD.mp4")

In [ ]:
df_video2 = df_video.astype(int)
df_video2 = pd.crosstab(df_video2['mean'], df_video2['std'])
sns.heatmap(df_video2)

In [ ]:
df_video.head(20)